# K-Means#

In [1]:

import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlens.ensemble import SuperLearner 

from itertools import combinations
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.kernel_approximation import RBFSampler
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.kernel_approximation import Nystroem

from sklearn.neural_network import MLPClassifier

from sklearn.cluster import KMeans


[MLENS] backend: threading


In [2]:
labels_f = pd.read_csv('../../data/labels_f_filtrado.csv')

In [3]:
datos = pd.read_csv('../../data/datos_filtrado.csv')

In [4]:
atributos = labels_f.columns.tolist()

In [5]:
atributos.remove('person')
atributos.remove('label')

In [6]:
y = labels_f['label']#.ravel()
X = labels_f.loc[:,atributos]

In [7]:
scaler = MaxAbsScaler().fit(X)
X = scaler.transform(X)

In [8]:
km = KMeans(n_clusters=10, init='k-means++', n_init=3, max_iter=300, precompute_distances= True, random_state=0)

In [9]:
km.fit(X)
X_means = km.transform(X)

In [10]:
def aplicarSmote(X_train_p, y_train_p, X_test):
    smote = SMOTE(ratio='minority')
    X_smt_train, y_smt_train = smote.fit_sample(X_train_p, y_train_p)
    #X_smt_train = pd.DataFrame(X_smt_train)
    #X_smt_train.columns = X_test.columns
    return (X_smt_train, y_smt_train)

In [11]:
def aplicarRus(X_train_p, y_train_p,X_test_p):
    rus= RandomUnderSampler(return_indices=True)
    #id_rus son los índices
    X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train_p, y_train_p)
    #X_rus_train = pd.DataFrame(X_rus_train)
    #X_rus_train.columns = X_test_p.columns
    return(X_rus_train, y_rus_train)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_means, y, test_size= 0.1, random_state= 123, stratify = y)

In [13]:
#X_train, y_train = aplicarRus(X_train,y_train)

In [14]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [15]:
def aplicarModelo(modelo,X_train_p, X_test_p, y_train_p, y_test_p, early = False):
    if(early):
        modelo.fit(X_train_p, y_train_p, early_stopping_rounds = 5, eval_set=[(X_test_p, y_test_p)])
    else: 
        modelo.fit(X_train_p, y_train_p)
    
    preds = modelo.predict(X_test_p)
    preds_prob = modelo.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, modelo.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    print()

In [34]:
#rbf_feature = RBFSampler(gamma=2,n_components= 300, random_state=123)
#X_features = rbf_feature.fit_transform(X)

In [ ]:
# fit the model and get the separating hyperplane using weighted classes
clf = svm.SVC(kernel='rbf', probability = True)


print('APLICO SMOTE')
aplicarModelo(clf,X_smt_train, X_test, y_smt_train, y_test, False)
print('APLICO RUS')
aplicarModelo(clf,X_rus_train, X_test, y_rus_train, y_test, False)

APLICO SMOTE


/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [16]:
knn = KNeighborsClassifier(n_neighbors=22)

print('APLICO SMOTE')
aplicarModelo(knn,X_smt_train, X_test, y_smt_train, y_test, False)
print('APLICO RUS')
aplicarModelo(knn,X_rus_train, X_test, y_rus_train, y_test, False)

APLICO SMOTE
train acurracy: 
0.8622362869198312
test acurracy: 
0.7492276004119465
Matriz de confusión: 
[[1395  449]
 [  38   60]]
Área bajo la curva: 
0.7416801319226172

APLICO RUS
train acurracy: 
0.7687074829931972
test acurracy: 
0.737384140061792
Matriz de confusión: 
[[1365  479]
 [  31   67]]
Área bajo la curva: 
0.7779173491522423



In [ ]:
nn = MLPClassifier( early_stopping=False, random_state=123)

print('APLICO SMOTE')
aplicarModelo(nn,X_smt_train, X_test, y_smt_train, y_test, False)
print('APLICO RUS')
aplicarModelo(nn,X_rus_train, X_test, y_rus_train, y_test, False)

APLICO SMOTE
